<h1> Tutorial 1.2 - Stochastic Solution of a Bimolecular Reaction </h1>

Here we examine a stochastic version of Tutorial 1.1.
<br/>

In Python you "import" libraries to be able to use their functionality.  The first several lines import certain functionality including certain operating system functions (os), standard numeric capabilities that are much like Matlab (numpy) and plotting capabilities (matplotlib). There lines are boiler-plate code for most pyLM scripts.<br/>

In order to use pyLM we need to import several libraries.  The first is pyLM proper (pyLM).  The second is a library with a number of functions such as ```nm()```, ```micron()```, ```ms()```, ```microsecond()```, etc. that allow cleaner definition of units.  Finally, we import the ```pyLM``` standard library of functionality ```pySTDLM```, which contains standard plotting and post-processing commands.


In [ ]:
# Import Standard Python Libraries
import os
import numpy as np
import matplotlib.pyplot as plt

# Import pyLM Libraries
from pyLM import *
from pyLM.units import *
from pySTDLM import *
from pySTDLM.PostProcessing import *

# Enable plotting inline in the Jupyter notebook
%matplotlib inline

In [ ]:
# Constants
V  = 1.0e-15       # L
NA = 6.022e23      # molecules/mole
kf = 1.07e5/(NA*V) # /M/s
kr = 0.351         # /s

<h2> Define a CME Simulation </h2>
<br/>
See the [pyLM Documentation](http://www.scs.illinois.edu/schulten/lm/documentation/index.html) <br/><br/>
We begin by creating a [CMESimulation](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_c_m_e_1_1_c_m_e_simulation.html) "object" that we call ```sim```. This object will include the definition of the whole stochastic simulation.

In [ ]:
# Create our CME simulation object
sim=CME.CMESimulation()

Next we define the chemical species with simulation. First. we specify the names of the chemical species.  Then we register these species with the simulation.  The [```defineSpecies()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_c_m_e_1_1_c_m_e_simulation.html#a7cb4a49c1096e57a7824358ec5cd2f57) function can be called multiple times and will add any new names to the list of species.


In [ ]:
# Define our chemical species
species = ['A', 'B', 'C']
sim.defineSpecies(species)

Here we add reactions to the simulation. We use the [```addReaction()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_c_m_e_1_1_c_m_e_simulation.html#a7a469a808184f548db1c853124c7eb68) function that is a member of the ```CMESimulation``` object. We add a bimolecular association reaction and a unimolecular dissociation reaction. When more than one reactant is involved, the list of reactant names should be passed as a tuple as can be seen in the reactant of the association reaction. The rates in this command must be in units of molecules and seconds, for instance units of ```/molecule/sec``` for the association reaction.


In [ ]:
# Add reactions to the simulation
sim.addReaction(reactant=('A','B'), product='C', rate=kf)
sim.addReaction(reactant='C', product=('A','B'), rate=kr)

Next, we add the initial particle counts to the simulation using the [```addParticles()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_c_m_e_1_1_c_m_e_simulation.html#a0c21058387ab0063399e16f6ddfa38df) function.

In [ ]:
# Set our initial species counts
sim.addParticles(species='A', count=1000)
sim.addParticles(species='B', count=1000)
sim.addParticles(species='C', count=0)

Finally, we define the simulation execution parameters. We have the simulation run for 30 seconds of real time saving results every 30 microseconds for a total of 1 million times. <br/>

Then we name the simulation output file and save the simulation definition to it.

In [ ]:
# Define simulation parameters: run for 10 seconds, saving data every ms
sim.setWriteInterval(microsecond(30))
sim.setSimulationTime(30)
filename = "T1.2-bimol.lm"
os.system("rm -rf %s"%(filename))
sim.save(filename)

In [ ]:
# Print simulation parameters to the notebook
sim

<h2> Run the Simulation </h2>
<br/>
Next we run the simulation. To do this, we specify which file has the problem specification (as saved two cells up). Lattice Microbes supports several different solvers for CME simulations; here we use the earliest algorithm called the Gillespie's algorithm (aka the Stochastic Simulation Algorithm). You can learn more [here](https://en.wikipedia.org/wiki/Gillespie_algorithm). <br/>

Because the CME represents a stochastic process, each instance of a simulation will have a slightly different trajectory.  Generally, we run many "replicate" simulations with the same or nearly the same starting conditions and then compute aggregate statistics over all of them.  Here, we only run a single replicate to demostrate the fluctuations.

In [ ]:
# Run 1 replicates using the Gillespie solver
sim.run(filename=filename, method="lm::cme::GillespieDSolver", replicates=1)

<h2> Post-Process Simulation </h2>
<br/>
See the [pySTDLM Documentation](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m.html)<br/>

Post-processing generally begins by getting a handle to the file. This is accomplished by passing the filename to the function [```openLMFile()```](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m_1_1_post_processing.html#a7a34458c6500be88081bec4c1396cbb8) which is supplied by the module [```PostProcessing```](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m_1_1_post_processing.html). This function does some error checking to make sure the file is generated by LM. <br/>

The function [```plotTrace```](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m_1_1_post_processing.html#afe08366f153c0518fe428b030047cdc6) will plot a list of species from the specified simulation replicate. The plot will appear in the Jupyter notebook because we previously specified ```%matplotlib inline```.<br/>

Finally, we close the LM File. It is very important that if you open an LM file with the function ```openLMFile``` that it be closed at the end of your post-processing with [```closeLMFile```](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m_1_1_post_processing.html#a94f59b9c78acbfa2885979aac45a1c71).   The function takes the filehandle that is returned by ```openLMFile``` as an argument. Also, not that once this function is called, any function that takes the filehandle as an argument will fail to work as the handle is now stale.  This is a common mistake and if you get crashing, check that you haven't prematurely closed the file.  This function is usually called last in a post-processing script.



In [ ]:
# Plot the solution
fh = openLMFile("T1.2-bimol.lm")
plotTrace(fh, species=['A','C'], replicate=1)
closeLMFile(fh)

<h2> Assignment 1 </h2>

1. Try plotting the average and variance of many replicates. You will find the documentation for pySTDLM useful. 

2. How many replicates are required to get a smooth average? How many for a smooth variance?

3. What happens to the noise when you add competing side reactions?

4. (Challenge) Can you derive an analytical solution for the system of equations?  Try fitting the rate constants using the results of the stochastic simulations. You may find [```scipy.optimize.curve_fit()```](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) useful for this.

